In [5]:
#legit bare meg som leker meg med dash bro, wtf
from dash import Dash, html, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px
from prophet.plot import plot_plotly, plot_components_plotly
from datetime import datetime
from prediktiv_analyse import prediktiv_analyse
from statistikk import statistikk
df = pd.read_csv(r'../data/lokasjonsdata.csv').rename(columns={'Unnamed: 0': 'Date'})



app = Dash()

app.layout = html.Div([
    html.H1(children='Heftig interaktiv visualisering', style={'textAlign': 'center'}),
    dcc.DatePickerSingle(
        id='Dato-velger',
        date=datetime.today()),
    dcc.Dropdown(
        options=[
                {'label': 'Visualisering', 'value': 'visualisering'},
                {'label': 'Statistikk', 'value': 'statistikk'},
                {'label': 'Prediktiv Analyse', 'value': 'prediktiv_analyse'}
            ],
        value='prediktiv_analyse',  # Default selection
        id='Plot-velger'),
    dcc.Dropdown(
        options=[{'label': col, 'value': col} for col in df.columns[1:]],  # Columns 1–4
        value=df.columns[1],  # Default selection
        id='By-velger'),
    dcc.Slider(1, 10, 1, value=1, id='slider'),
    
    dcc.Graph(id='graph-content'),
    
])

@callback(
    Output('graph-content', 'figure'),
    Input('Plot-velger', 'value'),
    Input('By-velger', 'value'),
    Input('slider', 'value')
    
)
def update_graph(plot, by, slider):
    if plot == 'visualisering':
        return px.line(df, x=df['Date'], y=df[by]) 
    elif plot == 'statistikk':
        return statistikk(by,df)
    elif plot == 'prediktiv_analyse':
        modell, forecast = prediktiv_analyse(slider, by, df[['Date', by]])
        return plot_plotly(modell, forecast)

In [ ]:
#skal brukes for å fremstille alt? hvis jeg tar meg tid til å gjøre det
app.run(jupyter_mode="inline",port=8050)

12:11:36 - cmdstanpy - INFO - Chain [1] start processing
12:11:37 - cmdstanpy - INFO - Chain [1] done processing
